Dataset: https://drryanmaness.wixsite.com/cyberconflict/cyber-conflict-dataset

- States: A, B
- Cyber incidents: A (initiator) -> B (target)
- Dyadic dataset: A <-> B

In [1]:
import pandas as pd

In [2]:
file_path = 'data/DCID.csv'
dcid = pd.read_csv(file_path)
print(dcid.head())

# cow_contries = pd.read_csv('data/COW-country-codes.csv')

   Cyberincidentnum  Dyadpair StateA  StateB                    Name  \
0                 1      2365     US  Russia  Regin malware campaign   
1                 1      2365     US  Russia             Duke Series   
2                 2      2365     US  Russia    QWERTY keystroke log   
3                 2      2365     US  Russia             Duke Series   
4                 3      2365     US  Russia           Duke Series_A   

  interactionstartdate  method  targettype  initiator  target  ...  Mdisplay-  \
0             01.02.08       3           2          2     365  ...        2.8   
1             08.04.08       4           2        365       2  ...        2.8   
2             01.02.08       4           2          2     365  ...        2.8   
3             09.04.08       4           2        365       2  ...        2.8   
4             08.04.08       4           2        365       2  ...        2.8   

   Musage-  Mil_Negative  Mpromise+  Mcoop+  Mconfid+  Mil_Positive  \
0      0.

In [ ]:
cow_countries = pd.read_csv('data/COW-country-codes.csv')

dcid = dcid.merge(cow_countries, left_on='initiator', right_on='CCode', how='left')
dcid = dcid.merge(cow_countries, left_on='target', right_on='CCode', how='left')

# USA = 2
# RUS = 365

dcid.rename(columns={'StateAbb_x': 'initiator_iso'}, inplace=True)
dcid.rename(columns={'StateAbb_y': 'target_iso'}, inplace=True)

dcid = dcid[['initiator_iso', 'target_iso']]

print(dcid.head())

In [ ]:
adjacency_matrix = pd.pivot_table(dcid, index='initiator_iso', columns='target_iso', aggfunc=len, fill_value=0)
print(adjacency_matrix)


In [ ]:
# adjacency_matrix.to_csv('adjacency_matrix.csv', index=True)


In [ ]:
import networkx as nx
from matplotlib import pyplot as plt

G = nx.DiGraph(adjacency_matrix)

G = nx.DiGraph()
G.add_edges_from(dcid.values)

plt.figure(figsize=(12, 6))
nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray', font_weight='bold')
plt.show()


In [ ]:
centrality = nx.eigenvector_centrality(G)
centrality = pd.DataFrame(centrality.items(), columns=['Country', 'Centrality'])
centrality = centrality.sort_values(by='Centrality', ascending=False)
print(centrality)
